In [1]:
import numpy as np
from tqdm import tqdm
from Coreset import *
from Kmeans import *
from sklearn.metrics import normalized_mutual_info_score as NMI

In [2]:
data = np.genfromtxt("data/bio_train.csv", delimiter=",")[:,3:]
labels_true = np.genfromtxt("data/bio_train.csv", delimiter=",")[:,0]

In [3]:
coreset = Coreset(data)
# print(coreset.X[0])
# print(coreset.mu)
# print(coreset.X[0] - coreset.mu)
coreset.get_q()
#print(np.linalg.norm(coreset.X))
#print(coreset.q[:10])
print(sum(coreset.q))

cs, rnd_indices = coreset.get_coreset(70000)
print(cs.shape)

145750it [00:00, -146454.28it/s]
0.9999999999999949
(70000, 74)


In [4]:
from itertools import product

eps = delta = np.arange(0.01, 1, 0.1)
 
for e, d in product(eps, delta):
    print(f'epsilon: {e:.2f}, delta: {d:.2f}, m: {coreset.get_m(eps=e, delta=d)}')
print(f'epsilon: 1, delta: 1, m: {coreset.get_m(eps=1, delta=1)}')


epsilon: 0.01, delta: 0.01, m: 569592233
epsilon: 0.01, delta: 0.11, m: 569568254
epsilon: 0.01, delta: 0.21, m: 569561787
epsilon: 0.01, delta: 0.31, m: 569557893
epsilon: 0.01, delta: 0.41, m: 569555097
epsilon: 0.01, delta: 0.51, m: 569552914
epsilon: 0.01, delta: 0.61, m: 569551124
epsilon: 0.01, delta: 0.71, m: 569549606
epsilon: 0.01, delta: 0.81, m: 569548288
epsilon: 0.01, delta: 0.91, m: 569547124
epsilon: 0.11, delta: 0.01, m: 4707373
epsilon: 0.11, delta: 0.11, m: 4707175
epsilon: 0.11, delta: 0.21, m: 4707122
epsilon: 0.11, delta: 0.31, m: 4707090
epsilon: 0.11, delta: 0.41, m: 4707066
epsilon: 0.11, delta: 0.51, m: 4707048
epsilon: 0.11, delta: 0.61, m: 4707034
epsilon: 0.11, delta: 0.71, m: 4707021
epsilon: 0.11, delta: 0.81, m: 4707010
epsilon: 0.11, delta: 0.91, m: 4707001
epsilon: 0.21, delta: 0.01, m: 1291592
epsilon: 0.21, delta: 0.11, m: 1291537
epsilon: 0.21, delta: 0.21, m: 1291523
epsilon: 0.21, delta: 0.31, m: 1291514
epsilon: 0.21, delta: 0.41, m: 1291508
epsil

## Epsilon and Delta Values chosen from the parameter search

epsilon, delta
(0.6, 0.5)
(0.7, 0.2)
(0.8, 0.7)
(0.9, 0.1)
(1, 1)


In [5]:
params = [
(0.6, 0.5),
(0.7, 0.2),
(0.8, 0.7),
(0.9, 0.1),
(1, 1)]

for eps, delta in params:
    print(f'epsilon: {eps:.2f}, delta: {delta:.2f}, m: {coreset.get_m(eps=eps, delta=delta)}')

epsilon: 0.60, delta: 0.50, m: 158209
epsilon: 0.70, delta: 0.20, m: 116237
epsilon: 0.80, delta: 0.70, m: 88992
epsilon: 0.90, delta: 0.10, m: 70317
epsilon: 1.00, delta: 1.00, m: 56954


KMeans with Coreset + KMeans with obtained values

In [6]:
coreset_sizes = [218972,153075,112991,86814,68783,56954]
for coreset_size in coreset_sizes:
    coreset = Coreset(data)
    coreset.get_q()
    cs, ind = coreset.get_coreset(coreset_size)

    model = Kmeans(k=153, max_iters=300)
    model.fit(cs)
    print(f"NMI KMeans+Coreset({coreset_size}) with true value: {NMI(labels_true[ind], model.labels_)}")

    model_next = Kmeans(k=153, max_iters=300)
    model_next.fit(data, cluster_initialization=ClusterInitialization.custom, custom_cluster_centers=model.centers)
    print(f"NMI KMeans with custom center and true value: {NMI(labels_true, model_next.labels_)}")
    print('---------------------------------------------------------------')

145750it [00:01, 115549.52it/s]
 88%|████████▊ | 263/300 [09:57<01:24,  2.27s/it]
Kmeans converged. Exiting loop.


100%|██████████| 300/300 [07:29<00:00,  1.50s/it]
12667it [00:00, 126663.48it/s]NMI KMeans with custom center and true value: 0.15960047053354545
---------------------------------------------------------------
145750it [00:01, 134233.80it/s]
100%|██████████| 300/300 [07:33<00:00,  1.51s/it]
11132it [00:00, 111314.27it/s]NMI KMeans with custom center and true value: 0.1601649409320816
---------------------------------------------------------------
145750it [00:01, 131049.89it/s]
100%|██████████| 300/300 [09:13<00:00,  1.85s/it]
6897it [00:00, 68958.89it/s]NMI KMeans with custom center and true value: 0.15918637817836773
---------------------------------------------------------------
145750it [00:01, 78576.00it/s]
  0%|          | 0/300 [00:00<?, ?it/s]
Kmeans converged. Exiting loop.

NMI KMeans+Coreset(86814) with true value: 0.24481132149239573
100%|██████████| 300/300 [